# 6. Constraints for Optimization Models

This notebook continues our step-by-step guide to creating optimization models. We are now at step 6 of our 7-step procedure:

1. ✓ Import Pyomo and Define the Sets
2. ✓ Define the Decision Variables
3. ✓ Define the Parameters
4. ✓ Define the Expressions
5. ✓ Define the Objective Function
6. ➡️ Define the Constraints

In this notebook, we focus on defining constraints - the rules that determine what solutions are feasible. We will explore:
- How to formulate different types of constraints
- Common constraint patterns
- Best practices for constraint definition
- Ways to structure complex constraint systems

## Setup: Creating Our Model Environment

Let's continue with our production planning example:

In [1]:
# Create model
import pyomo.environ as pyo 

m = pyo.ConcreteModel()

# Sets
m.I = pyo.Set(initialize=['P1', 'P2', 'P3'])  # Products
m.J = pyo.Set(initialize=['M1', 'M2'])        # Machines

# Variables  
m.x = pyo.Var(m.I, domain=pyo.NonNegativeReals, initialize={'P1': 10, 'P2': 20, 'P3': 30})      # Production quantities
m.y = pyo.Var(m.I, m.J, domain=pyo.Binary, initialize={(i, j): 0 for i in m.I for j in m.J})           # Machine assignments

# Parameters
m.p = pyo.Param(m.I, initialize={'P1': 100, 'P2': 150, 'P3': 200})  # Product prices
m.c = pyo.Param(m.J, initialize={'M1': 50, 'M2': 40})               # Machine costs
m.d = pyo.Param(m.I, initialize={'P1': 80, 'P2': 120, 'P3': 60})    # Demands
m.cap = pyo.Param(m.J, initialize={'M1': 160, 'M2': 200})           # Machine capacities

# Common expressions
m.revenue = pyo.Expression(expr=sum(m.p[i] * m.x[i] for i in m.I))
m.production_cost = pyo.Expression(expr=sum(m.c[j] * m.y[i,j] for i in m.I for j in m.J))

## Basic Constraint Definition

In Pyomo, constraints are mathematical relationships that must be satisfied by the solution. They can be:
- Inequality constraints (<=, >=) 
- Equality constraints (==)
- Range constraints (bounds)
  
For each of these constraints, the structure is the same: 
- the set it applies to, 
- the function handler or rule, and 
- the expression.

An example of a constraint is:

```python
m.capacity = pyo.Constraint(
    m.J,
    rule=lambda m, j: sum(m.x[i] for i in m.I) <= m.cap[j]
)
```

Here, 

- `m.J` is the set it applies to (it can be set of machines, products, etc.)
- `lambda m, j` is the function handler or rule (it basically means that for each machine `j` in the set `m.J`, the formula `lambda` is applied)
- `sum(m.x[i] for i in m.I) <= m.cap[j]` is the logical expression
    - `sum(m.x[i] for i in m.I)` is the left-hand side of the expression
    - `m.cap[j]` is the right-hand side of the expression
    - `<=` is the inequality operator

Let's look at an example:

In [2]:
# Basic capacity constraint
m.capacity = pyo.Constraint(
    m.J,
    rule=lambda m, j: sum(m.x[i] for i in m.I) <= m.cap[j]
)

We can print each constraint by accessing the constraint object directly:

In [3]:
print(m.capacity['M1'].expr)

x[P1] + x[P2] + x[P3]  <=  160


Or iterate over the constraints set:

In [4]:
for j in m.J:
    print(j, m.capacity[j].expr)
    print("The value of the constraint is:", m.capacity[j]())


M1 x[P1] + x[P2] + x[P3]  <=  160
The value of the constraint is: 60
M2 x[P1] + x[P2] + x[P3]  <=  200
The value of the constraint is: 60


Note that the value of the constraint is the left-hand side of the constraint (here, we have not solved the model yet and only evaluated the constraints at the initial value of the variables).


Let's look at another example:

In [5]:
# Demand satisfaction constraint  
m.demand = pyo.Constraint(
    m.I,
    rule=lambda m, i: m.x[i] >= m.d[i]
)

print(m.demand['P1'].expr)
for i in m.I:
    print(i, m.demand[i].expr)
    print("The value of the constraint is:", m.demand[i]())


80  <=  x[P1]
P1 80  <=  x[P1]
The value of the constraint is: 10
P2 120  <=  x[P2]
The value of the constraint is: 20
P3 60  <=  x[P3]
The value of the constraint is: 30


Another way to display the constraints is to use the `pprint()` method:

In [6]:
m.demand.pprint()

demand : Size=3, Index=I, Active=True
    Key : Lower : Body  : Upper : Active
     P1 :  80.0 : x[P1] :  +Inf :   True
     P2 : 120.0 : x[P2] :  +Inf :   True
     P3 :  60.0 : x[P3] :  +Inf :   True


Let's see one more example:

In [7]:
# Machine allocation constraint
m.allocation = pyo.Constraint(
    m.I, m.J,
    rule=lambda m, i, j: m.x[i] <= m.cap[j] * m.y[i,j]
)

m.allocation.pprint()

allocation : Size=6, Index=I*J, Active=True
    Key          : Lower : Body                 : Upper : Active
    ('P1', 'M1') :  -Inf : x[P1] - 160*y[P1,M1] :   0.0 :   True
    ('P1', 'M2') :  -Inf : x[P1] - 200*y[P1,M2] :   0.0 :   True
    ('P2', 'M1') :  -Inf : x[P2] - 160*y[P2,M1] :   0.0 :   True
    ('P2', 'M2') :  -Inf : x[P2] - 200*y[P2,M2] :   0.0 :   True
    ('P3', 'M1') :  -Inf : x[P3] - 160*y[P3,M1] :   0.0 :   True
    ('P3', 'M2') :  -Inf : x[P3] - 200*y[P3,M2] :   0.0 :   True


With this, we have a complete model:
- We have a set that contains the group of entities
- We have decision variables that represent what we want to optimize over
- We have parameters that represent the data of the model
- We have expressions that are useful for defining objective, constraints, output metrics, etc.
- We have set the objective function and whether it is a maximization or minimization
- We have constraints that represent the rules that must be satisfied by the solution

A complete model does not mean it guarantees a solution. It means that the model is fully defined and ready to be solved. Whether it will find a solution depends on the solver and whether the problem is feasible. A feasible problem is one that has at least one feasible solution, i.e. a solution that satisfies all the constraints.

In some cases, we can have a setting where the model is not feasible, perhaps due to a conflict in the constraints or simply impossible to satisfy all the constraints at once. In such cases, we can try to relax some of the constraints or change the model to make it feasible. We will cover this in the verification adn validation (V&V) step of the optimization process. Regardless of the feasibility of the model, we can still use the model to explore the solution space and understand the problem better.

That said, let's see how to solve the model. We can use the `SolverFactory` class with the solver of our choice to solve the model and call the `solve()` method on the model object. Let's use the `glpk` solver for this example:

In [8]:
solver = pyo.SolverFactory('glpk')
results = solver.solve(m)

print(results)


Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 10
  Number of nonzeros: 21
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.026021718978881836



In this example, we have an infeasible solution (since we created a dummy model). Let's create an example of a feasible model and solve it:

In [9]:
m = pyo.ConcreteModel()

m.I = pyo.Set(initialize=['P1', 'P2', 'P3'])
m.J = pyo.Set(initialize=['M1', 'M2'])
m.x = pyo.Var(m.I, m.J, domain=pyo.NonNegativeReals, initialize={(i, j): 0 for i in m.I for j in m.J})
m.y = pyo.Var(m.J, domain=pyo.Binary, initialize={(j): 0 for j in m.J})
m.p = pyo.Param(m.I, initialize={'P1': 100, 'P2': 150, 'P3': 200})
m.c = pyo.Param(m.J, initialize={'M1': 50, 'M2': 40})
m.d = pyo.Param(m.I, initialize={'P1': 80, 'P2': 120, 'P3': 60})
m.cap = pyo.Param(m.J, initialize={'M1': 160, 'M2': 200})

m.revenue = pyo.Expression(expr=sum(m.p[i] * m.x[i, j] for i in m.I for j in m.J))
m.production_cost = pyo.Expression(expr=sum(m.c[j] * m.x[i,j] for i in m.I for j in m.J))

m.obj = pyo.Objective(expr=m.revenue - m.production_cost, sense=pyo.maximize)
m.capacity = pyo.Constraint(m.J, rule=lambda m, j: sum(m.x[i,j] for i in m.I) <= m.cap[j])
m.demand = pyo.Constraint(m.I, rule=lambda m, i: sum(m.x[i,j] for j in m.J) >= m.d[i])
m.assignment = pyo.Constraint(m.I, m.J, rule=lambda m, i, j: m.x[i,j] <= m.cap[j] * m.y[j])

solver = pyo.SolverFactory('glpk')
results = solver.solve(m)

print(results)


Problem: 
- Name: unknown
  Lower bound: 42000.0
  Upper bound: 42000.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 8
  Number of nonzeros: 24
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.026836156845092773
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In this example, we change the setting. The decision variables are now defined as `m.x[i,j]` and `m.y[j]`, where `x[i,j]` is the production quantity of product `i` on machine `j`, and `y[j]` is the binary variable that indicates whether machine `j` is used. 

The constraints are now defined as:
- `m.capacity` is the capacity constraint for each machine
- `m.demand` is the demand constraint for each product
- `m.assignment` is the assignment constraint for each product and machine, which ensures that the production quantity of each product on each machine is less than or equal to the capacity of the machine multiplied by the binary variable that indicates whether the machine is used

The objective function is now defined as the difference between the revenue and the production cost, which is the total revenue minus the total production cost.

We can now solve the model and access the solution:

In [10]:
solver = pyo.SolverFactory('glpk')
results = solver.solve(m)

print(results)


Problem: 
- Name: unknown
  Lower bound: 42000.0
  Upper bound: 42000.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 8
  Number of nonzeros: 24
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 5
      Number of created subproblems: 5
  Error rc: 0
  Time: 0.024807214736938477
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In this example, we have a feasible solution. We can now access the solution and evaluate the objective function and constraints at the solution:

In [11]:
print("The optimal solution is: x =", [(i, j, m.x[i,j]()) for i in m.I for j in m.J])
print("The optimal solution is: y =", [(j, m.y[j]()) for j in m.J])
print("The optimal objective value is: ", m.obj())

The optimal solution is: x = [('P1', 'M1', 0.0), ('P1', 'M2', 80.0), ('P2', 'M1', 120.0), ('P2', 'M2', 0.0), ('P3', 'M1', 40.0), ('P3', 'M2', 120.0)]
The optimal solution is: y = [('M1', 1.0), ('M2', 1.0)]
The optimal objective value is:  42000.0


Note that the optimal solution is now different from the initial solution. This is because the solver has found a better solution that satisfies all the constraints and maximizes the objective function.

Since we define expressions, we can also evaluate the expressions at the solution:

In [12]:
print("The optimal revenue is: ", m.revenue())
print("The optimal production cost is: ", m.production_cost())


The optimal revenue is:  58000.0
The optimal production cost is:  16000.0


We can also evaluate the constraints at the solution:

In [13]:
print("The capacity constraint status: ", [(j, m.capacity[j]() <= m.cap[j]) for j in m.J])
print("The demand constraint status: ", [(i, m.demand[i]() >= m.d[i]) for i in m.I])
print("The assignment constraint status: ", [(i, j, m.assignment[i,j]() <= m.cap[j] * m.y[j]()) for i in m.I for j in m.J])


The capacity constraint status:  [('M1', True), ('M2', True)]
The demand constraint status:  [('P1', True), ('P2', True), ('P3', True)]
The assignment constraint status:  [('P1', 'M1', True), ('P1', 'M2', True), ('P2', 'M1', True), ('P2', 'M2', True), ('P3', 'M1', True), ('P3', 'M2', True)]


## Conclusion

In this notebook, we have learned how to define constraints in Pyomo. We have seen how to define different types of constraints and how to use them in the model. We have also seen how to display and evaluate the constraints at a given solution.